# Simulation and Noise Models

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.insert(0, '../')

We can set up a noise model, following the [Qiskit textbook](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html#Correcting-errors-in-qubits). We define our noise model to have equal chances of X and Z:

In [6]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise_model(p_err):

    error_gate1 = pauli_error([("X", p_err / 2), ("Z", p_err / 2), ("I", 1 - p_err)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "id")
    return noise_model

We set up a test harness to run through some examples:

In [7]:
import sys
import os

sys.path.insert(0, ".." + os.sep)
from benchmarking.benchmarking_tools import SurfaceCodeBenchmarkingTool
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from surface_code.fitters import GraphDecoder
from surface_code.circuits import SurfaceCodeLogicalQubit
from qiskit import QuantumCircuit, execute, QuantumRegister, ClassicalRegister, Aer
from tqdm import tqdm

import multiprocessing as mp

In [ ]:
logical = []
noise = [5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3, 1e-2, 2e-2]
d = 3
T = 1

qubit = SurfaceCodeLogicalQubit(d)
qubit.stabilize()
qubit.identity_data()
qubit.stabilize()
qubit.readout_z()
qubit.draw(output='mpl', fold=50)

In [ ]:
benchmarking_tool = SurfaceCodeBenchmarkingTool(
    decoder=GraphDecoder(d=d, T=T),
    readout_circuit=qubit,
    noise_model_func=get_noise_model,
)

print("\nSIMULATE: (d={},T={})\n".format(benchmarking_tool.d, benchmarking_tool.T))
correct_logical_value = 0
noise_values = [
    5e-5,
    1e-4,
    2e-4,
    5e-4,
    1e-3,
    2e-3,
    4e-3,
    5e-3,
    6e-3,
    7e-3,
    8e-3,
    9e-3,
    1e-2,
    2e-2,
]
benchmarking_tool.simulate_readout(
    correct_logical_value=correct_logical_value, noise_values=noise_values, shots=100000, save_data=False
)
print("Done!")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for log_plot in [True, False]:
    fig = plt.figure(figsize=(3.5, 2.5), dpi=200)
    ax = fig.subplots()
    benchmarking_tool.plot_benchmark_data(
        fig=fig,
        ax=ax,
        label="d={},T={}".format(benchmarking_tool.d, benchmarking_tool.T),
        log=log_plot,
    )
    plt.plot(
        benchmarking_tool.benchmark_data["noise"],
        0.5 * np.array(benchmarking_tool.benchmark_data["noise"]),
        "--",
        label="breakeven",
    )
    plt.legend(loc="lower right", prop={"size": 6})
    ax.set_xlabel("Physical Error Rate", size=10)
    ax.set_ylabel("Logical Error Rate", size=10)
    ax.set_title("Surface Code Performance", size=10)
    fig.tight_layout()
    plt.show()

So, we see the d=3, T=1 surface code does quite well for this particular error model!

In [ ]:
logical = []
noise = [5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3, 1e-2, 2e-2]
d = 3
T = 1

qubit = SurfaceCodeLogicalQubit(d)
qubit.stabilize()
qubit.identity()
qubit.stabilize()
qubit.readout_z()
benchmarking_tool_full = SurfaceCodeBenchmarkingTool(
    decoder=GraphDecoder(d=d, T=T),
    readout_circuit=qubit,
    noise_model_func=get_noise_model,
)

print("\nSIMULATE: (d={},T={})\n".format(benchmarking_tool_full.d, benchmarking_tool_full.T))
correct_logical_value = 0
noise_values = [
    5e-5,
    1e-4,
    2e-4,
    5e-4,
    1e-3,
    2e-3,
    4e-3,
    5e-3,
    6e-3,
    7e-3,
    8e-3,
    9e-3,
    1e-2,
    2e-2,
]
benchmarking_tool_full.simulate_readout(
    correct_logical_value=correct_logical_value, noise_values=noise_values, shots=100000, save_data=False
)
print("Done!")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for log_plot in [True, False]:
    fig = plt.figure(figsize=(3.5, 2.5), dpi=200)
    ax = fig.subplots()
    benchmarking_tool_full.plot_benchmark_data(
        fig=fig,
        ax=ax,
        label="d={},T={}".format(benchmarking_tool_full.d, benchmarking_tool_full.T),
        log=log_plot,
    )
    plt.plot(
        benchmarking_tool_full.benchmark_data["noise"],
        np.array(benchmarking_tool_full.benchmark_data["noise"]),
        "--",
        label="breakeven",
    )
    plt.legend(loc="lower right", prop={"size": 6})
    ax.set_xlabel("Physical Error Rate", size=10)
    ax.set_ylabel("Logical Error Rate", size=10)
    ax.set_title("Surface Code Performance", size=10)
    fig.tight_layout()
    plt.show()